In [ ]:
%pip install pandas numpy tqdm scikit-learn

In [ ]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
import sys
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

In [ ]:
df = pd.read_csv('./Data/23-24-problem_logs.csv')

In [ ]:
df.sort_values(by=['user_xid','skill_id','start_time'], inplace=True)

In [ ]:
wins = []
fails = []
for name, group in tqdm(df.groupby(by=['user_xid', 'skill_id'])):
    w = 0
    f = 0
    for idx, row in group.iterrows():
        wins.append(w)
        fails.append(f)
        if row['discrete_score'] == 1:
            w += 1
        else:
            f += 1
             
    

In [ ]:
df['wins'] = wins
df['fails'] = fails

In [ ]:
skills = np.sort(df.skill_id.unique())
n_s = len(skills)

In [ ]:
res_arr = sp.dok_array((df.shape[0], 3*n_s))
i = 0
for idx, row in tqdm(df.iterrows()):
    #user_idx = np.where(users == row['user_xid'])[0][0]
    skill_idx = np.where(skills == row['skill_id'])[0][0]
    # res_arr[i,user_idx] = 1
    # res_arr[i,problem_idx] = 1
    res_arr[i,skill_idx] = 1
    res_arr[i,n_s+skill_idx] = row['wins']
    res_arr[i,2*n_s+skill_idx] = row['fails']
    i += 1

In [ ]:
res_arr_coo = res_arr.tocoo()
y = df.discrete_score.to_numpy(copy=True)
fn = './Data/23-24-problem_logs'
fn_X = fn + '_X.npz'
fn_y = fn + '_y.npy'
sp.save_npz(fn_X, res_arr_coo)
np.save(fn_y, y)

In [ ]:
model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10**4)

In [ ]:
model.fit(res_arr_coo,y)

In [ ]:
y_pred = model.predict_proba(res_arr_coo)[:,1]

In [ ]:
log_loss(y,y_pred)

In [ ]:
roc_auc_score(y,y_pred)